Pablo Jimeno - UPV/EHU 2016

In [ ]:
%run 'Utilities_main.ipynb'

In [ ]:
#-------------------------------------------------------------------
def import_Y_A_results(subsample_bins):
    """Import results"""    
    
    sample_metas = []
    Y_vals = []
    A_vals = []
    Y_errs = []
    A_errs = []
    
    for sub_bin in subsample_bins:
          
        rich_min, rich_max, z_min, z_max = sub_bin
        sample_tag = sub_tag(sub_bin)
        
        if (rich_min>=rich_max) or (z_min>=z_max):
            continue
    
        #-------------------------------------------------------------------
        ## Import meta & SZ data:       
        store_Y_A_file = '%s/SZ_values/Y_A_data_%s.pickle' % (data_dir, sample_tag)
        with open(store_Y_A_file, 'rb') as handle:
            store_Y_A_data = pickle.load(handle)
        
        #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
        sub_meta = store_Y_A_data[0]
        Y        = store_Y_A_data[1]
        A        = store_Y_A_data[2]
        Y_err    = store_Y_A_data[3]
        A_err    = store_Y_A_data[4]
        
        sample_metas.append(sub_meta)
        Y_vals.append(Y)
        A_vals.append(A)
        Y_errs.append(Y_err)
        A_errs.append(A_err)
                    
    sample_metas = np.array(sample_metas)
    Y_vals = np.array(Y_vals)
    A_vals = np.array(A_vals)
    Y_errs = np.array(Y_errs)
    A_errs = np.array(A_errs)
                
    return sample_metas, Y_vals, A_vals, Y_errs, A_errs
    

In [ ]:
#-------------------------------------------------------------------
def model_z_evolution(Y_vals, Y_err_vals, z_vals):
    
    # Define cosmology:
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
    
    def Da(z): return cosmo.comoving_transverse_distance(z).value
    def E(z): return cosmo.efunc(z)
    def x(z): return 
    
    def z_evolution(z, norm, alpha):
        
        alpha_evo = (1. + z)**(alpha)
        E2_scale = E(z)**(2./3.)
        Da_scale = 1./Da(z)**2.
        #Delta_scale = 18.*np.pi**2 + 82.*x(z) - 39.*x(z)**2
        
        z_scale = norm*E2_scale*alpha_evo*Da_scale
        
        return z_scale
           
    alpha_guess = -1.
    params_guess = np.array([Y_vals[0]*Da(z_vals[0])**2, alpha_guess])
    
    params_fit, params_cov = optimize.curve_fit(z_evolution, z_vals, Y_vals, sigma=Y_err_vals, 
                                                p0=params_guess, absolute_sigma=True)
    
    norm_val = params_fit[0]
    alpha_val = params_fit[1]
    alpha_err = np.sqrt(np.diag(params_cov))[1]
    
    z_vector = np.arange(0.1, 0.6, 0.001)
    Y_scale_vals = z_evolution(z_vector, norm_val, alpha_val)
    
    return z_vector, Y_scale_vals, alpha_val, alpha_err
    
    
#-------------------------------------------------------------------
def model_R_scale(Y_vals, Y_err_vals, R_vals):
    
    # Define cosmology:
    cosmo = FlatLambdaCDM(H0=70, Om0=0.3)
       
    def R_scaling(R, norm, MR_slope):
        
        beta = 2./3.
        MT_scale = R**(MR_slope*(1. + beta))
        
        R_scale = norm*MT_scale
        
        return R_scale
           
    MR_slope_guess = 1.1
    params_guess = np.array([Y_vals[1], MR_slope_guess])
    
    params_fit, params_cov = optimize.curve_fit(R_scaling, R_vals, Y_vals, sigma=Y_err_vals, 
                                                p0=params_guess, absolute_sigma=True)
    
    norm_val = params_fit[0]
    MR_slope_val = params_fit[1]
    MR_slope_err = np.sqrt(np.diag(params_cov))[1]
    
    R_vector = np.arange(10., 150., 1.)
    Y_scale_vals = R_scaling(R_vector, norm_val, MR_slope_val)
    
    return R_vector, Y_scale_vals, MR_slope_val, MR_slope_err
   
    
#-------------------------------------------------------------------
def model_evolution(sample_metas, Y_vals, A_vals, Y_errs, A_errs, nr_bins, nz_bins):
       
    ## Plot results:
    markersize = 3.
    colors = ['k', 'b', 'r', 'g', 'c', 'y']
    SZ_y_lims = [0.05, 150.]
    
    
    def plot_param_vs_z(ax, param, legend_loc, log=False):
        
        ax.set_xlim([0.1, 0.6])
        if log: ax.set_yscale("log")
        ax.set_xlabel(r'z')
                
        for r in range(nr_bins-1, -1, -1):
        
            color = colors[r]
            i = r*nz_bins
            j = (r+1)*nz_bins

            z = sample_metas[i:j, 6]
            N = sample_metas[i:j, 4]
            Y = Y_vals[i:j]
            A = A_vals[i:j]
            Y_err = Y_errs[i:j]
            A_err = A_errs[i:j]
            z_vector, Y_scale_vals, alpha_val, alpha_err = model_z_evolution(Y, Y_err, z)
            label = r'$\lambda$: [%i, %i), $\alpha$=%.1f$\pm$%.1f' % (sample_metas[i+1][0], sample_metas[i+1][1],
                                                              alpha_val, alpha_err)
               
            if param=='Y':
                ax.set_ylabel(r'$\left<Y\right>$')
                ax.set_ylim(SZ_y_lims)
                ax.errorbar(z, Y, yerr=Y_err, fmt='%s:'%color, marker='o', markersize=markersize, label=label)
                ax.plot(z_vector, Y_scale_vals, color=color, alpha=0.3)
                
            if param=='A':
                ax.set_ylabel(r'$\left<A\right>$')
                ax.errorbar(z, A, yerr=A_err, fmt='%s:'%color, marker='o', markersize=markersize, label=label)
                
        if param=='Y': ax.plot([],[], alpha=0.3, label=r'E(z)$^{2/3}$ (1+z)$^{\alpha}$ Da(z)$^{-2}$')
        ax.legend(loc=legend_loc, numpoints=1)
        
        
    def plot_param_vs_R(ax, param, legend_loc, log=False):
        
        ax.set_xlim([sample_metas[:,5].min() - 10, sample_metas[:,5].max() + 10])
        if log: ax.set_yscale("log")
        ax.set_xlabel(r'$\lambda$')
                
        for z in range(nz_bins):
        
            color = colors[z]

            R = sample_metas[z::nz_bins, 5]
            N = sample_metas[z::nz_bins, 4]
            Y = Y_vals[z::nz_bins]
            A = A_vals[z::nz_bins]
            Y_err = Y_errs[z::nz_bins]
            A_err = A_errs[z::nz_bins]
            R_vector, Y_scale_vals, MR_slope_val, MR_slope_err = model_R_scale(Y, Y_err, R)
            label = 'z: [%.3f, %.3f), $\gamma_{M|\lambda}$=%.2f$\pm$%.2f' % (sample_metas[z][2], sample_metas[z][3],
                                                                             MR_slope_val, MR_slope_err)
            
            if param=='Y':
                ax.set_ylabel(r'$\left<Y\right>$')
                ax.set_ylim(SZ_y_lims)
                ax.errorbar(R, Y, yerr=Y_err, fmt='%s:'%color, marker='o', markersize=markersize, label=label)
                ax.plot(R_vector, Y_scale_vals, color=color, alpha=0.3)
            if param=='A':
                ax.set_ylabel(r'$\left<A\right>$')
                ax.errorbar(R, A, yerr=A_err, fmt='%s:'%color, marker='o', markersize=markersize, label=label)
                
        if param=='Y': ax.plot([],[], alpha=0.3, label=r'$\lambda^{\gamma_{M|\lambda}}$')
        ax.legend(loc=legend_loc, numpoints=1)
        
        
    def plot_alpha_evolution(ax):
        
        ax.set_xlabel(r'$\lambda$')
        ax.set_ylabel(r'$\alpha$')
        
        alpha_vals = []
        alpha_errs = []
        R_vals = []
        
        for r in range(nr_bins-1, -1, -1):
        
            i = r*nz_bins
            j = (r+1)*nz_bins

            z = sample_metas[i:j, 6]
            R = sample_metas[i:j, 5].mean()
            Y = Y_vals[i:j]
            Y_err = Y_errs[i:j]
            z_vector, Y_scale_vals, alpha_val, alpha_err = model_z_evolution(Y, Y_err, z)
            alpha_vals.append(alpha_val)
            alpha_errs.append(alpha_err)
            R_vals.append(R)
                           
        ax.errorbar(R_vals, alpha_vals, yerr=alpha_errs, fmt='k:', marker='o', markersize=markersize)                      

    #------------------------------------------------------------------- 
    #fig = plt.figure(figsize=(5.5, 4))
    #sub_meta : [rich_min, rich_max, z_min, z_max, N_clu, rich_mean, z_mean]
    
    #ax1 = fig1.add_subplot(221)
    #plot_param_vs_z(ax1, 'Y (SZ model)', 1, log=True)
    
    #ax2 = fig1.add_subplot(222)
    #plot_param_vs_z(ax2, 'A (dust model)', 2)
    
    #ax3 = fig1.add_subplot(223)
    #plot_param_vs_R(ax3, 'Y (SZ model)', 2, log=True)
    
    #ax4 = fig1.add_subplot(224)
    #plot_param_vs_R(ax4, 'A (dust model)', 2)
           
    ## Display figure:
    #plot_file = '%s/SZ_modeling/Values_evolution.png' % plot_dir
    #fig1.savefig(plot_file, bbox_inches='tight')
    #save_figure(fig1, 'Values_evolution')
    #plt.close()
    
    
    #-------------------------------------------------------------------
    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(111)
    plot_param_vs_z(ax, 'Y (SZ model)', 1, log=True)
               
    ## Display figure:
    plot_file = '%s/SZ_modeling/z_evolution.png' % plot_dir
    fig.savefig(plot_file, bbox_inches='tight')
    save_figure(fig, 'z_evolution')
        
    #-------------------------------------------------------------------
    fig = plt.figure(figsize=(5, 4))
    ax = fig.add_subplot(111)
    plot_param_vs_R(ax, 'Y (SZ model)', 2, log=True)
           
    ## Display figure:
    plot_file = '%s/SZ_modeling/R_scaling.png' % plot_dir
    fig.savefig(plot_file, bbox_inches='tight')
    save_figure(fig, 'R_scaling')
      
    #-------------------------------------------------------------------
    fig = plt.figure(figsize=(5, 4))   
    ax = fig.add_subplot(111)
    plot_alpha_evolution(ax)
               
    ## Display figure:
    plot_file = '%s/SZ_modeling/Alpha_evolution.png' % plot_dir
    fig.savefig(plot_file, bbox_inches='tight')
    plt.close()
    
    
    
    
#-------------------------------------------------------------------

#R_vals = [20, 30, 50, 100, 200]
#R_vals = [20, 300]
#R_vals = [20, 30, 45, 70, 100, 150, 300]

#z_vals = [0.100, 0.250, 0.400, 0.550]
#z_vals = [0.080, 0.550]

subsample_bins, nr_bins, nz_bins = automatic_binning(R_vals, z_vals) 

## Run code:
def run_evolution_code():
    sample_metas, Y_vals, A_vals, Y_errs, A_errs = import_Y_A_results(subsample_bins)
    model_evolution(sample_metas, Y_vals, A_vals, Y_errs, A_errs, nr_bins, nz_bins)
    
    
try:
    run_evolution_code()
except:
    %run 'Modelling_SZ.ipynb'
    run_evolution_code()

print '\nEvolution analysis completed.'

In [ ]:
redm